<a href="https://colab.research.google.com/github/AvantiShri/colab_notebooks/blob/master/revcomp/CreateSiameseBinaryModelFromStandardModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%tensorflow_version 1.x

import keras

def get_binary_reg_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Convolution1D(
              input_shape=(1000,4), nb_filter=16, filter_length=15))
    model.add(keras.layers.normalization.BatchNormalization())
    model.add(keras.layers.core.Activation("relu"))
    model.add(keras.layers.convolutional.Convolution1D(
            nb_filter=16, filter_length=14))
    model.add(keras.layers.normalization.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.convolutional.Convolution1D(
            nb_filter=16, filter_length=14))
    model.add(keras.layers.normalization.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.convolutional.MaxPooling1D(pool_length=40,
                                                      strides=40))         
    model.add(keras.layers.Flatten())
    model.add(keras.layers.core.Dense(output_dim=1, trainable=True,
                                    init="glorot_uniform"))
    model.add(keras.layers.core.Activation("sigmoid"))
    model.compile(optimizer = keras.optimizers.Adam(lr=0.001), 
                  loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [6]:
model = get_binary_reg_model()
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(1000, 4), filters=16, kernel_size=15)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=16, kernel_size=14)`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=16, kernel_size=14)`
  app.launch_new_instance()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 986, 16)           976       
_________________________________________________________________
batch_normalization_4 (Batch (None, 986, 16)           64        
_________________________________________________________________
activation_4 (Activation)    (None, 986, 16)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 973, 16)           3600      
_________________________________________________________________
batch_normalization_5 (Batch (None, 973, 16)           64        
_________________________________________________________________
activation_5 (Activation)    (None, 973, 16)           0         
__________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(strides=40, pool_size=40)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(trainable=True, units=1, kernel_initializer="glorot_uniform")`


In [8]:
#Create a sub-model that extracts the logit
binary_model_getlogits = keras.models.Model(inputs=model.inputs,
                                            outputs=model.layers[-2].output)

In [9]:
binary_model_getlogits.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4_input (InputLayer)  (None, 1000, 4)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 986, 16)           976       
_________________________________________________________________
batch_normalization_4 (Batch (None, 986, 16)           64        
_________________________________________________________________
activation_4 (Activation)    (None, 986, 16)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 973, 16)           3600      
_________________________________________________________________
batch_normalization_5 (Batch (None, 973, 16)           64        
_________________________________________________________________
activation_5 (Activation)    (None, 973, 16)           0   

In [12]:
#create the siamese model

fwd_sequence_input = keras.layers.Input(shape=(1000,4))
rev_sequence_input = keras.layers.Lambda(function=lambda x: x[:,::-1,::-1])(fwd_sequence_input)
fwd_logit_output = binary_model_getlogits(fwd_sequence_input)
rev_logit_output = binary_model_getlogits(rev_sequence_input)
average_logits = keras.layers.Average()([fwd_logit_output, rev_logit_output])
sigmoid_out = keras.layers.Activation("sigmoid")(average_logits)

siamese_model = keras.models.Model(inputs=[fwd_sequence_input],
                                   outputs=[sigmoid_out])

In [16]:
#test it out

import numpy as np

random_sequence = np.random.RandomState(1234).random((10,1000,4))

fwd_logits = binary_model_getlogits.predict(random_sequence)
rev_logits = binary_model_getlogits.predict(random_sequence[:,::-1,::-1])
siamese_fwd_logits = siamese_model.predict(random_sequence)
siamese_rev_logits = siamese_model.predict(random_sequence[:,::-1,::-1])

In [26]:
from scipy.special import expit
expit(0.5*(fwd_logits + rev_logits)) - siamese_fwd_logits

array([[ 0.0000000e+00],
       [-5.9604645e-08],
       [ 0.0000000e+00],
       [-5.9604645e-08],
       [ 0.0000000e+00],
       [ 5.9604645e-08],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00]], dtype=float32)

In [21]:
siamese_fwd_logits - siamese_rev_logits

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)